In [10]:
import os
from dotenv import load_dotenv
import openai
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient
from azure.search.documents.models import QueryType, Vector

load_dotenv()
# Replace these with your own values, either in environment variables or directly here
AZURE_STORAGE_ACCOUNT = os.environ.get("AZURE_STORAGE_ACCOUNT")
AZURE_STORAGE_CONTAINER = os.environ.get("AZURE_STORAGE_CONTAINER")
AZURE_SEARCH_SERVICE = os.environ.get("AZURE_SEARCH_SERVICE")
AZURE_SEARCH_INDEX = os.environ.get("AZURE_SEARCH_INDEX") 
AZURE_OPENAI_SERVICE = os.environ.get("AZURE_OPENAI_SERVICE") 
AZURE_OPENAI_GPT_DEPLOYMENT = os.environ.get("AZURE_OPENAI_GPT_DEPLOYMENT") 
AZURE_OPENAI_CHATGPT_DEPLOYMENT = os.environ.get("AZURE_OPENAI_CHATGPT_DEPLOYMENT") 
AZURE_OPENAI_EMB_DEPLOYMENT = os.environ.get("AZURE_OPENAI_EMB_DEPLOYMENT")
AZURE_SEARCH_SERVICE_KEY = os.environ.get("AZURE_SEARCH_SERVICE_KEY") 

KB_FIELDS_CONTENT = os.environ.get("KB_FIELDS_CONTENT") or "content"
KB_FIELDS_CATEGORY = os.environ.get("KB_FIELDS_CATEGORY") or "category"
KB_FIELDS_SOURCEPAGE = os.environ.get("KB_FIELDS_SOURCEPAGE") or "sourcepage"


# Used by the OpenAI SDK
openai.api_base = f"https://{AZURE_OPENAI_SERVICE}.openai.azure.com"
openai.api_version = "2023-06-01-preview"
openai.api_key = os.environ.get("AZURE_OPENAI_API_KEY") 
# Comment these two lines out if using keys, set your API key in the OPENAI_API_KEY environment variable and set openai.api_type = "azure" instead
openai.api_type = "azure"

# Set up clients for Cognitive Search and Storage
search_client = SearchClient(
    endpoint=f"https://{AZURE_SEARCH_SERVICE}.search.windows.net",
    index_name=AZURE_SEARCH_INDEX,
    credential=AzureKeyCredential(AZURE_SEARCH_SERVICE_KEY))

In [11]:
# ChatGPT uses a particular set of tokens to indicate turns in conversations
prompt_prefix = """<|im_start|>system
어시스턴트는 에 정보가 충분하지 않은 경우에는 모른다고 말하검색 결과에서 나온 문서에 대한 질문을 도와줍니다. 답변은 간결하게 작성하세요.

표 형식의 정보는 HTML 테이블로 반환합니다. 마크다운 형식은 반환하지 마세요.
각 출처에는 이름 뒤에 콜론과 실제 정보가 있으며, 응답에 사용하는 각 사실에 대한 출처 이름을 항상 포함하세요. 소스를 참조할 때는 대괄호를 사용합니다(예: [info1.txt]). 소스를 결합하지 말고 각 소스를 개별적으로 나열합니다(예: [info1.txt][info2.pdf]).
Sources:
{sources}

<|im_end|>"""

turn_prefix = """
<|im_start|>user
"""

turn_suffix = """
<|im_end|>
<|im_start|>assistant
"""

prompt_history = turn_prefix

history = []

summary_prompt_template = """아래는 지금까지의 대화 요약과 사용자가 검색결과에서 검색하여 답변해야 하는 새로운 질문입니다. 대화와 새 질문을 기반으로 검색 쿼리를 생성합니다.

Summary:
{summary}

Question:
{question}

Search query:
"""

In [12]:
# Execute this cell multiple times updating user_input to accumulate chat history
user_input = "인터넷 형광펜 서비스 소개좀 부탁해요"

# Exclude category, to simulate scenarios where there's a set of docs you can't see
exclude_category = None

if len(history) > 0:
    completion = openai.Completion.create(
        engine=AZURE_OPENAI_GPT_DEPLOYMENT,
        prompt=summary_prompt_template.format(summary="\n".join(history), question=user_input),
        temperature=1,
        max_tokens=32,
        stop=["\n"])
    search = completion.choices[0].text
else:
    search = user_input

# Use Azure OpenAI to compute an embedding for the query
query_vector = openai.Embedding.create(engine=AZURE_OPENAI_EMB_DEPLOYMENT, input=search)["data"][0]["embedding"]


print("Searching:", search)
print("-------------------")
filter = "category ne '{}'".format(exclude_category.replace("'", "''")) if exclude_category else None

# Hybrid search with semantic search and vector reranking

r = search_client.search(search, 
                         filter=filter,
                         query_type=QueryType.SEMANTIC, 
                         query_language="en-us", 
                         query_speller="lexicon", 
                         semantic_configuration_name="default", 
                         top=3,
                         vector=Vector(value=query_vector, k=50, fields="embedding") if query_vector else None)

# Vector search only
# r = search_client.search(search, top=3, vector=Vector(value=query_vector, k=50, fields="embedding") if query_vector else None)

results = [doc[KB_FIELDS_SOURCEPAGE] + ": " + doc[KB_FIELDS_CONTENT].replace("\n", "").replace("\r", "") for doc in r]
content = "\n".join(results)

prompt = prompt_prefix.format(sources=content) + prompt_history + user_input + turn_suffix

print("\n-------------------\n".join(history))
print("\n-------------------\nPrompt:\n" + prompt)

completion = openai.Completion.create(
    engine=AZURE_OPENAI_CHATGPT_DEPLOYMENT, 
    prompt=prompt, 
    temperature=0.7, 
    max_tokens=1024,
    stop=["<|im_end|>", "<|im_start|>"])

prompt_history += user_input + turn_suffix + completion.choices[0].text + "\n<|im_end|>" + turn_prefix
history.append("user: " + user_input)
history.append("assistant: " + completion.choices[0].text)

print("\n-------------------\n".join(history))




Searching: 인터넷 형광펜 서비스 소개좀 부탁해요
-------------------


-------------------
Prompt:
<|im_start|>system
어시스턴트는 에 정보가 충분하지 않은 경우에는 모른다고 말하검색 결과에서 나온 문서에 대한 질문을 도와줍니다. 답변은 간결하게 작성하세요.

표 형식의 정보는 HTML 테이블로 반환합니다. 마크다운 형식은 반환하지 마세요.
각 출처에는 이름 뒤에 콜론과 실제 정보가 있으며, 응답에 사용하는 각 사실에 대한 출처 이름을 항상 포함하세요. 소스를 참조할 때는 대괄호를 사용합니다(예: [info1.txt]). 소스를 결합하지 말고 각 소스를 개별적으로 나열합니다(예: [info1.txt][info2.pdf]).
Sources:
dbr_sample-3.pdf: 이중칭찬릴레이커뮤니티‘티키타카’ , 간단하게자료 를전달 하 는클라 우 드‘아이센드’ 등8개서비스 를론칭했다.은라이너는그중3번째서비스다.김대표는평소활 자중 독수준으로읽을것을늘손에쥐고 있었다.그러다중요한내용을발견하면바로 바지에서형광펜을꺼내줄을그었다.김대표 는인터넷에는형광펜이없는게늘아쉬웠다. 검색시장에도불편함을느꼈다.과제를위해 검색하다상위에뜬페이지를살펴봐도원하 정보가없는경우가허다했다.문득누군가정는 말중요한정보를형광펜으로표시해주면좋겠 다는생각을했다.그러기위해선우선인터넷 에서사 할수있는형광펜이필요했다. 용그렇게2015년7월형광펜기능이탑재된 iOS브라우저앱‘라이너’를만들었다.브라우79 DBR No. 370Downloaded by news123(DIGITAL), sh89.kang@kt.com, 2023-06-02 출처:동아닷컴캡처구글웹스토어,마이크로소프트엣지추가기능페이지에서라이너를설치하고,저장하고싶은텍스트를 드래그하면형광펜기능이활성화된다.여러색을적용해정보의유형,중요도등을구분하고,해당정보는 라이너웹사이트,앱에서모아볼수있다저에서정보를탐색하다중요한텍스트를드래 그하면형광펜버튼이활성화되는식이다.하 이라이트한정보들 는을한데모아다시검색하 불편함없이찾아볼수있게

In [13]:
history.clear()